In [25]:
import numpy as np
import pandas as pd

import torch
from torch import nn

from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
import cv2

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from pathlib import Path
import glob

In [26]:
data_dir = Path('/kaggle/input/chest-xray-pneumonia/chest_xray')
train_dir = data_dir / 'train'
test_dir = data_dir / 'test'
val_dir = data_dir / 'val'

In [27]:
class PneumoniaDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

In [38]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.permute(1, 2, 0))
])

train_data = PneumoniaDataset(data_dir=train_dir,
                              transform=transform)

test_data = PneumoniaDataset(data_dir=test_dir,
                             transform=transform)

val_data = PneumoniaDataset(data_dir=val_dir,
                            transform=transform)

In [39]:
from torch.utils.data import Subset

subset = Subset(train_data, list(range(5)))

for item in subset:
    print(item[0].shape)

torch.Size([224, 224, 3])
torch.Size([224, 224, 3])
torch.Size([224, 224, 3])
torch.Size([224, 224, 3])
torch.Size([224, 224, 3])


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
